# 带密码保护功能的锁

书中是这样说的：“第 3 章介绍的两个锁实现都假设只有持有锁的客户端会调用 release() 方法来解锁，但实际上其他客户端即使没有成功加锁，也可以通过指定相同的锁键并执行 release() 方法来解锁。为了避免出现没有持有锁的客户端解锁这种情况，可以给锁加上密码保护功能，使锁只在给定正确密码的情况下才会被解锁。”但我觉得本地代码的逻辑应该只有拿到锁，才能释放锁，你没拿到锁，逻辑上就不会执行 release()。

之前我在找工作背八股时，涉及到 Redis 分布式锁的问题，下面是我当时的笔记，我感觉下面的逻辑可能更匹配需要带“密码保护”功能的锁。
> 分布式锁，一般会依托第三方组件进行实现，Redis 是用的最多的，主要分为加锁和解锁：  
> 1）加锁使用 `set key value nx ex seconds` 命令（`setnx`），其中加时间是为了防止如果获取锁的服务挂掉了，那么锁永远得不到释放；value 要用持有者的 id，比如 UUID，也就是要加个 owner，释放时要看一下是不是自己的名字，因为要满足谁申请，谁释放的原则，防止比如 A 拿到了锁，锁过期了，然后 B 抢到了锁；此时 A 执行完了，释放了锁，但 B 还没有执行完。  
> 2）解锁，我们首先要看看一下是不是该线程的 id，如果是则删除，因为涉及到两步，所以要用 Lua 脚本实现原子化。为了防止，比如判断完是自己的，你准备删时，锁过期了，另一个线程拿到了锁，这时你就会又误删别人的锁。

总之上面两种描述，最终的目的是一样的，就是避免一个客户端去释放别的客户端的锁。下面解释一下作者的代码。

1. 获取锁键的值（也就是加锁时设置的密码）。
2. 检查锁键的值是否与给定的密码相同，如果相同就执行第 3 步，否则执行第 4 步。
3. 删除锁键并返回 True 表示解锁成功。（保证只有拿锁的客户端可以解锁）
4. 不对锁键做任何动作，只返回 False 表示解锁失败。

这里的 `WATCH` 和事务是为了，防止在检查密码通过后，准备删除时，锁的密码被改了。另外这里的事务只有一条命令需要开事务吗？需要的，因为 `WATCH` 只有在事务中才会发挥作用。

下面 Python 的写法还挺特别的，像是 Lua 脚本一样。这种情况最好还是直接用 Lua，能够减少客户端与服务器的交互，提升性能。像下面用 `WATCH` 就需要多次客户端与服务器的交互。

另外 Redis 的乐观锁不会面临ABA问题。

In [ ]:
class IdentityLock:

    def __init__(self, client, key):
        self.client = client
        self.key = key

    def acquire(self, password):
        """
        尝试获取一个带有密码保护功能的锁，
        成功时返回True，失败时则返回False。
        password参数用于设置上锁/解锁密码。
        """
        return self.client.set(self.key, password, nx=True) is True

    def release(self, password):
        """
        根据给定的密码，尝试释放锁。
        锁存在并且密码正确时返回True，
        返回False则表示密码不正确或者锁已不存在。
        """
        tx = self.client.pipeline()
        try:
            # 监视锁键以防它发生变化
            tx.watch(self.key)
            # 获取锁键存储的密码
            lock_password = tx.get(self.key)
            # 比对密码
            if lock_password == password:
                # 情况1：密码正确，尝试解锁
                tx.multi()
                tx.delete(self.key)
                return tx.execute()[0]==1  # 返回删除结果
            else:
                # 情况2：密码不正确
                tx.unwatch()
        except WatchError:
            # 尝试解锁时发现键已变化
            pass
        finally:
            # 确保连接正确回归连接池，redis-py的要求
            tx.reset()
        # 密码不正确或者尝试解锁时失败
        return False

作者这里并没有实现带自动解锁功能的锁，他说会让代码变得很复杂，有兴趣的读者可自行尝试。